In [36]:
import pandas as pd
import numpy as np
from Process_Functions import normalized_time_axis
import copy
original_data = pd.read_csv('data_120.csv')

# 清理数据，删除所有值均为空的列
data_cleared = original_data.dropna(axis=1, how='all')

# 切片不同的序列
data_sliced = []
column_num = len(data_cleared.columns)

for i in range(0, column_num, 2):
    df_sliced = data_cleared.iloc[:, i:i+2].copy()  # 使用 copy() 避免警告
    df_sliced.columns = ['t' + str(i // 2 + 1), 'i' + str(i // 2 + 1)]
    
    # 删除所有值为空的行
    new_df_sliced = df_sliced.dropna(axis=0, how="all").copy()
    
    # 确保 new_df_sliced 不是空的再进行操作
    if not new_df_sliced.empty:
        new_df_sliced.loc[:, new_df_sliced.columns[0]] -= new_df_sliced.iloc[0, 0]
    
    data_sliced.append(new_df_sliced)
    
    
def normalized_translocation(data,threshold=450,drop_head=5,drop_tail=-5):
    translocation_region=data[data.iloc[:,1]<threshold]
    translocation_region=translocation_region.iloc[drop_head:drop_tail,:]
    translocation_region.iloc[:,0]=translocation_region.iloc[:,0]-translocation_region.iloc[0,0]
    
    return translocation_region


print(data_cleared)


             Time           I       Time.1         I.1       Time.2  \
0      803.099976  689.697266  3448.310059  698.242188  8688.639648   
1      803.109985  687.866211  3448.320068  700.683594  8688.650391   
2      803.119995  690.917969  3448.330078  702.514648  8688.660156   
3      803.130005  687.255859  3448.340088  697.021484  8688.669922   
4      803.140015  686.645508  3448.350098  696.411133  8688.679688   
...           ...         ...          ...         ...          ...   
54232         NaN         NaN          NaN         NaN          NaN   
54233         NaN         NaN          NaN         NaN          NaN   
54234         NaN         NaN          NaN         NaN          NaN   
54235         NaN         NaN          NaN         NaN          NaN   
54236         NaN         NaN          NaN         NaN          NaN   

              I.2        Time.3         I.3        Time.4         I.4  ...  \
0      694.580078  13283.610352  693.359375  16853.890625  698.852539

In [22]:
data_sliced[107]

,t108.0,i108.0
0,0.00000,698.242188
1,0.00000,696.411133
2,0.00000,697.021484
3,0.03125,701.904297
4,0.03125,693.969727
...,...,...
14097,140.96875,695.190430
14098,140.96875,692.138672
14099,140.96875,695.190430
14100,141.00000,694.580078


In [23]:
translocation_data=[]
for series in data_sliced:
    translocation_series=normalized_translocation(series,threshold=min(series.iloc[:,1])+60,drop_head=5)
    translocation_data.append(translocation_series)
    

In [24]:
import copy
def normalized_time_axis(data):
    normalized_translocation_region=copy.deepcopy(data)
    normalized_translocation_region.iloc[:,0]=(normalized_translocation_region.iloc[:,0]-min(normalized_translocation_region.iloc[:,0]))/(max(normalized_translocation_region.iloc[:,0])-min(normalized_translocation_region.iloc[:,0]))
    return normalized_translocation_region
data_processed=[normalized_time_axis(series) for series in translocation_data]

In [35]:
data_sliced[107]

,t108.0,i108.0
0,0.00000,698.242188
1,0.00000,696.411133
2,0.00000,697.021484
3,0.03125,701.904297
4,0.03125,693.969727
...,...,...
14097,140.96875,695.190430
14098,140.96875,692.138672
14099,140.96875,695.190430
14100,141.00000,694.580078


In [26]:
import statistics
max_length = statistics.mean([series.shape[0] for series in processed_data])
max_length

1150.7333333333333

In [27]:
max_length = max([df.shape[0] for df in data_processed])
for i, df in enumerate(data_processed):
    if df.shape[0] < max_length:
        num_missing = max_length - df.shape[0]
        missing_df = pd.DataFrame(index=range(num_missing), columns=df.columns)
        data_processed[i] = pd.concat([df, missing_df], ignore_index=True)

# 水平拼接所有 DataFrame
combined_df = pd.concat(data_processed, axis=1)


In [34]:
combined_df.to_excel('data_120_try.xlsx', index=False)